In [1]:
!pip install datasets
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 243 kB 38.6 MB/s 
     |████████████████████████████████| 118 kB 52.3 MB/s 
     |████████████████████████████████| 3.3 MB 5.2 MB/s 
     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
     |████████████████████████████████| 636 kB 49.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.15
    Uninstalling huggingface-hub-0.0.15:
      Successfully uninstalled huggingface-hub-0.0.15


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("csv", data_files = {"train": "/content/drive/MyDrive/Colab Notebooks/train_tweets.csv",
                                            "test": "/content/drive/MyDrive/Colab Notebooks/test_tweets.csv"})

Using custom data configuration default-533b6bc04c269785


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-533b6bc04c269785/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], max_length = 64, padding = "max_length", truncation = True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [6]:
from torch.utils.data import DataLoader
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
train_dataset = tokenized_datasets["train"].shuffle(seed = 42)
eval_dataset = tokenized_datasets["test"].shuffle(seed = 42)
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = 32)
eval_dataloader = DataLoader(eval_dataset, batch_size = 32)

In [7]:
def average(batch, attention_mask):
  original_length = 0
  while original_length < len(attention_mask[0]) and attention_mask[0][original_length]:
    original_length += 1

  sentence_embeddings = torch.mean(batch[0][1:original_length], 0, True)

  for i in range(1, len(batch)):
    original_length = 0
    while original_length < len(attention_mask[i]) and attention_mask[i][original_length]:
      original_length += 1
    sentence_embeddings = torch.cat((sentence_embeddings, torch.mean(batch[i][1:original_length], 0, True)), 0)

  return sentence_embeddings  

In [20]:
def hierarchical_avg_max(sentence, mask, window_size):
  original_length = 0
  while original_length < len(mask) and mask[original_length]:
    original_length += 1

  original_window = original_length-window_size+1

  if original_window > 2:
    sentence = torch.reshape(sentence, (sentence.shape[0], 1, sentence.shape[1]))
    window = sentence[0]
    for i in range(1, window_size):
      window = torch.cat((window, sentence[i]), 0)
    windows = torch.mean(window, 0, True)

    for i in range(1, sentence.shape[0] - window_size + 1):
      window = sentence[i]
      for j in range(i + 1, i + window_size):
        window = torch.cat((window, sentence[j]), 0)
      windows = torch.cat((windows, torch.mean(window, 0, True)), 0)

    sentence_embeddings = torch.reshape(torch.max(windows[1:original_window-1], 0, True)[0], (1, sentence.shape[-1]))
  else:
    sentence_embeddings = torch.mean(sentence[1:original_length-1], 0, True)

  return sentence_embeddings

In [21]:
def hierarchical_max_avg(sentence, mask, window_size):
  original_length = 0
  while original_length < len(mask) and mask[original_length]:
    original_length += 1

  original_window = original_length-window_size+1

  if original_window > 2:
    sentence = torch.reshape(sentence, (sentence.shape[0], 1, sentence.shape[1]))
    window = sentence[0]
    for i in range(1, window_size):
      window = torch.cat((window, sentence[i]), 0)
    windows = torch.max(window, 0, True)[0]

    for i in range(1, sentence.shape[0] - window_size + 1):
      window = sentence[i]
      for j in range(i + 1, i + window_size):
        window = torch.cat((window, sentence[j]), 0)
      windows = torch.cat((windows, torch.max(window, 0, True)[0]), 0)

    sentence_embeddings = torch.reshape(torch.mean(windows[1:original_window-1], 0, True), (1, sentence.shape[-1]))
  else:
    sentence_embeddings = torch.max(sentence[1:original_length-1], 0, True)[0]

  return sentence_embeddings

In [22]:
def hierarchical(batch, attention_mask, prio = "avg", window_size = 2):
  if prio == "avg":
    hierarchical_prio = hierarchical_avg_max
  elif prio == "max":
    hierarchical_prio = hierarchical_max_avg

  sentence_embeddings = hierarchical_prio(batch[0], attention_mask[0], window_size)
  for i in range(1, len(batch)):
    sentence_embeddings = torch.cat((sentence_embeddings, hierarchical_prio(batch[i], attention_mask[i], window_size)), 0)

  return sentence_embeddings

In [23]:
import torch.nn as nn
from transformers import BertModel
import torch
class BertClassifier(nn.Module):
    def __init__(self, pretrained:str, freeze_bert = False, num_classes = 2, pooling_strategy = "cls", pooling_output_layer = -1, hierarchical_window_size = 2):
        super(BertClassifier, self).__init__()
        D_in, H, D_out = 768, 50, num_classes
        self.bert = BertModel.from_pretrained(pretrained)
        self.classifier = nn.Sequential(nn.Linear(D_in, H), nn.ReLU(), nn.Linear(H, D_out))
        self.pooling_strategy = pooling_strategy
        self.pooling_output_layer = pooling_output_layer
        self.hierarchical_window_size = hierarchical_window_size

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask, output_hidden_states = True):
        outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask, output_hidden_states = output_hidden_states)
        
        if self.pooling_strategy == "cls":
          sentence_embeddings = outputs[1]
        else:
          batch = outputs[2][self.pooling_output_layer]
          if self.pooling_strategy == "average":
            sentence_embeddings = average2(batch, attention_mask)
          elif self.pooling_strategy == "hierarchical_avg_max":
            sentence_embeddings = hierarchical(batch, attention_mask, "avg", self.hierarchical_window_size)
          elif self.pooling_strategy == "hierarchical_max_avg":
            sentence_embeddings = hierarchical(batch, attention_mask, "max", self.hierarchical_window_size)

        logits = self.classifier(sentence_embeddings.to(device))
        return logits      

In [36]:
model = BertClassifier("dbmdz/bert-base-turkish-cased", num_classes = 3, pooling_strategy = "hierarchical_max_avg", pooling_output_layer = -2, hierarchical_window_size = 3)  

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
loss_fn = nn.CrossEntropyLoss()

In [38]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 5e-5)

In [39]:
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps = 0, num_training_steps = num_training_steps)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [41]:
from tqdm.auto import tqdm
import numpy as np
import time
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*70)
    t0_epoch, t0_batch = time.time(), time.time()
    model.train()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    for step, batch in enumerate(train_dataloader):
        batch_counts +=1
        b_input_ids = batch["input_ids"].to(device)
        b_attn_mask = batch["attention_mask"].to(device)
        b_labels = batch["labels"].type(torch.LongTensor).to(device)
        model.zero_grad()     
        logits = model(b_input_ids, b_attn_mask)
        loss = loss_fn(logits, b_labels)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

    avg_train_loss = total_loss / len(train_dataloader)
    print("-"*70)

    val_loss = []
    val_accuracy = []
    model.eval()
    for batch in eval_dataloader:
        b_input_ids = batch["input_ids"].to(device)
        b_attn_mask = batch["attention_mask"].to(device)
        b_labels = batch["labels"].type(torch.LongTensor).to(device)
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        preds = torch.argmax(logits, dim=1).flatten()
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
        
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    time_elapsed = time.time() - t0_epoch
    print(f"{epoch + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
    print("-"*70)
    print("\n")

print("Training complete!")

  0%|          | 0/866 [00:00<?, ?it/s]

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.031598   |     -      |     -     |   25.47  
   1    |   40    |   0.948126   |     -      |     -     |   24.17  
   1    |   60    |   0.858688   |     -      |     -     |   24.22  
   1    |   80    |   0.727426   |     -      |     -     |   24.18  
   1    |   100   |   0.733482   |     -      |     -     |   24.08  
   1    |   120   |   0.673703   |     -      |     -     |   24.23  
   1    |   140   |   0.705702   |     -      |     -     |   24.31  
   1    |   160   |   0.661012   |     -      |     -     |   24.19  
   1    |   180   |   0.690568   |     -      |     -     |   24.16  
   1    |   200   |   0.698089   |     -      |     -     |   24.22  
   1    |   220   |   0.653583   |     -      |     -     |   24.34  
   1    |   240   |   0.614122   |     -      |     -     |   24.27  
   1    |   260   |

KeyboardInterrupt: ignored